<a href="https://colab.research.google.com/github/majid9418/majid/blob/main/multi_class_BreakHis2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
print(tf.__version__)

In [ ]:
!pip install tensorflow

In [ ]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

In [ ]:
import os

# Define the path where you want to save the dataset in your Google Drive
dataset_path = '/content/drive/MyDrive/breakhis/BreaKHis_v1.tar.gz'

# Make sure the directory exists
os.makedirs(os.path.dirname(dataset_path), exist_ok=True)

# Download the dataset
!wget -O {dataset_path} http://www.inf.ufpr.br/vri/databases/BreaKHis_v1.tar.gz

--2024-09-15 10:16:01--  http://www.inf.ufpr.br/vri/databases/BreaKHis_v1.tar.gz
Resolving www.inf.ufpr.br (www.inf.ufpr.br)... 200.17.202.113, 2801:82:80ff:8001:216:ccff:feaa:79
Connecting to www.inf.ufpr.br (www.inf.ufpr.br)|200.17.202.113|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://www.inf.ufpr.br/vri/databases/BreaKHis_v1.tar.gz [following]
--2024-09-15 10:16:03--  https://www.inf.ufpr.br/vri/databases/BreaKHis_v1.tar.gz
Connecting to www.inf.ufpr.br (www.inf.ufpr.br)|200.17.202.113|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4273561758 (4.0G) [application/octet-stream]
Saving to: ‘/content/drive/MyDrive/breakhis/BreaKHis_v1.tar.gz’

/content/drive/MyDr 100%[===================>]   3.98G  18.3MB/s    in 3m 54s  

2024-09-15 10:19:58 (17.4 MB/s) - ‘/content/drive/MyDrive/breakhis/BreaKHis_v1.tar.gz’ saved [4273561758/4273561758]



In [ ]:
import tarfile

# Extract the dataset
with tarfile.open(dataset_path, 'r:gz') as tar:
    tar.extractall(path='/content/drive/MyDrive/breakhis/')

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB5
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, LearningRateScheduler
from sklearn.utils.class_weight import compute_class_weight
import numpy as np
import matplotlib.pyplot as plt
import os
import shutil
import glob
from sklearn.model_selection import train_test_split

# Define base directory
base_dir = '/content/drive/MyDrive/breakhis/BreaKHis_v1/histology_slides/breast'

# Define subdirectories for benign and malignant
benign_dir = os.path.join(base_dir, 'benign', 'SOB')
malignant_dir = os.path.join(base_dir, 'malignant', 'SOB')

# Create directories for training and validation data
train_dir = os.path.join(base_dir, 'train')
val_dir = os.path.join(base_dir, 'validation')

os.makedirs(train_dir, exist_ok=True)
os.makedirs(val_dir, exist_ok=True)

# Create subdirectories for benign and malignant in train and validation folders
benign_subclasses = ['adenosis', 'fibroadenoma', 'phyllodes_tumor', 'tubular_adenoma']
malignant_subclasses = ['ductal_carcinoma', 'lobular_carcinoma', 'mucinous_carcinoma', 'papillary_carcinoma']

for subclass in benign_subclasses + malignant_subclasses:
    os.makedirs(os.path.join(train_dir, subclass), exist_ok=True)
    os.makedirs(os.path.join(val_dir, subclass), exist_ok=True)

# Function to split data and copy to respective directories
def split_and_copy_images(source_dir, train_dest_dir, val_dest_dir, split_ratio=0.2):
    image_files = glob.glob(os.path.join(source_dir, '**', '*.png'), recursive=True)

    # Debugging: Print the number of images found
    print(f"Found {len(image_files)} images in {source_dir}")

    if len(image_files) == 0:
        return

    train_files, val_files = train_test_split(image_files, test_size=split_ratio, random_state=42)

    # Debugging: Print the number of images in training and validation sets
    print(f"Training set size: {len(train_files)}, Validation set size: {len(val_files)}")

    for file in train_files:
        shutil.copy(file, train_dest_dir)

    for file in val_files:
        shutil.copy(file, val_dest_dir)

# Split and copy images for each subclass in benign
for subclass in benign_subclasses:
    benign_sub_dir = os.path.join(benign_dir, subclass)
    split_and_copy_images(benign_sub_dir, os.path.join(train_dir, subclass), os.path.join(val_dir, subclass))

# Split and copy images for each subclass in malignant
for subclass in malignant_subclasses:
    malignant_sub_dir = os.path.join(malignant_dir, subclass)
    split_and_copy_images(malignant_sub_dir, os.path.join(train_dir, subclass), os.path.join(val_dir, subclass))

train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.3,
    zoom_range=0.3,
    horizontal_flip=True,
    rotation_range=40,
    width_shift_range=0.3,
    height_shift_range=0.3
)

validation_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

validation_generator = validation_datagen.flow_from_directory(
    val_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)


Found 444 images in /content/drive/MyDrive/breakhis/BreaKHis_v1/histology_slides/breast/benign/SOB/adenosis
Training set size: 355, Validation set size: 89
Found 1014 images in /content/drive/MyDrive/breakhis/BreaKHis_v1/histology_slides/breast/benign/SOB/fibroadenoma
Training set size: 811, Validation set size: 203
Found 453 images in /content/drive/MyDrive/breakhis/BreaKHis_v1/histology_slides/breast/benign/SOB/phyllodes_tumor
Training set size: 362, Validation set size: 91
Found 569 images in /content/drive/MyDrive/breakhis/BreaKHis_v1/histology_slides/breast/benign/SOB/tubular_adenoma
Training set size: 455, Validation set size: 114
Found 3451 images in /content/drive/MyDrive/breakhis/BreaKHis_v1/histology_slides/breast/malignant/SOB/ductal_carcinoma
Training set size: 2760, Validation set size: 691
Found 626 images in /content/drive/MyDrive/breakhis/BreaKHis_v1/histology_slides/breast/malignant/SOB/lobular_carcinoma
Training set size: 500, Validation set size: 126
Found 792 images

**Additional augmentation** for underrepresented subclasses:

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB5
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, LearningRateScheduler, ReduceLROnPlateau
from sklearn.utils.class_weight import compute_class_weight
import numpy as np
import matplotlib.pyplot as plt
import os
import shutil
import glob
from sklearn.model_selection import train_test_split
import imgaug.augmenters as iaa

# Define base directory
base_dir = '/content/drive/MyDrive/breakhis/BreaKHis_v1/histology_slides/breast'

# Define subdirectories for benign and malignant
benign_dir = os.path.join(base_dir, 'benign', 'SOB')
malignant_dir = os.path.join(base_dir, 'malignant', 'SOB')

# Create directories for training and validation data
train_dir = os.path.join(base_dir, 'train')
val_dir = os.path.join(base_dir, 'validation')

os.makedirs(train_dir, exist_ok=True)
os.makedirs(val_dir, exist_ok=True)

# Create subdirectories for benign and malignant in train and validation folders
benign_subclasses = ['adenosis', 'fibroadenoma', 'phyllodes_tumor', 'tubular_adenoma']
malignant_subclasses = ['ductal_carcinoma', 'lobular_carcinoma', 'mucinous_carcinoma', 'papillary_carcinoma']

for subclass in benign_subclasses + malignant_subclasses:
    os.makedirs(os.path.join(train_dir, subclass), exist_ok=True)
    os.makedirs(os.path.join(val_dir, subclass), exist_ok=True)

# Function to split data and copy to respective directories
def split_and_copy_images(source_dir, train_dest_dir, val_dest_dir, split_ratio=0.2):
    image_files = glob.glob(os.path.join(source_dir, '**', '*.png'), recursive=True)

    # Debugging: Print the number of images found
    print(f"Found {len(image_files)} images in {source_dir}")

    if len(image_files) == 0:
        return

    train_files, val_files = train_test_split(image_files, test_size=split_ratio, random_state=42)

    # Debugging: Print the number of images in training and validation sets
    print(f"Training set size: {len(train_files)}, Validation set size: {len(val_files)}")

    for file in train_files:
        shutil.copy(file, train_dest_dir)

    for file in val_files:
        shutil.copy(file, val_dest_dir)

# Split and copy images for each subclass in benign
for subclass in benign_subclasses:
    benign_sub_dir = os.path.join(benign_dir, subclass)
    split_and_copy_images(benign_sub_dir, os.path.join(train_dir, subclass), os.path.join(val_dir, subclass))

# Split and copy images for each subclass in malignant
for subclass in malignant_subclasses:
    malignant_sub_dir = os.path.join(malignant_dir, subclass)
    split_and_copy_images(malignant_sub_dir, os.path.join(train_dir, subclass), os.path.join(val_dir, subclass))

# General data augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    brightness_range=[0.9, 1.1],
)

val_datagen = ImageDataGenerator(rescale=1./255)

# Create general data generators
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

validation_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

# Calculate the number of images in each subclass
subclass_counts = {subclass: len(glob.glob(os.path.join(train_dir, subclass, '*.png'))) for subclass in benign_subclasses + malignant_subclasses}
print(subclass_counts)

# Determine underrepresented classes
mean_count = np.mean(list(subclass_counts.values()))
underrepresented_classes = [subclass for subclass, count in subclass_counts.items() if count < mean_count]
print(f"Underrepresented classes: {underrepresented_classes}")

# Custom augmentation pipeline for underrepresented classes
aug_pipeline = iaa.Sequential([
    iaa.Fliplr(0.5),
    iaa.Flipud(0.2),
    iaa.Affine(rotate=(-45, 45)),
    iaa.Multiply((0.8, 1.2)),
    iaa.GaussianBlur(sigma=(0.0, 3.0)),
    iaa.AdditiveGaussianNoise(scale=(0.01*255, 0.05*255))
])

# Augment and add images for underrepresented classes
for subclass in underrepresented_classes:
    subclass_dir = os.path.join(train_dir, subclass)
    images = glob.glob(os.path.join(subclass_dir, '*.png'))

    for img_path in images:
        img = tf.keras.preprocessing.image.load_img(img_path)
        img_array = tf.keras.preprocessing.image.img_to_array(img)
        img_array = np.expand_dims(img_array, axis=0)

        for _ in range(int(mean_count / len(images))):
            aug_images = aug_pipeline(images=img_array.astype(np.uint8))
            for aug_img in aug_images:
                aug_img_path = os.path.join(subclass_dir, f"aug_{os.path.basename(img_path)}")
                aug_img = tf.keras.preprocessing.image.array_to_img(aug_img)
                aug_img.save(aug_img_path)

# Re-create generators after augmentation
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

validation_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)


Found 444 images in /content/drive/MyDrive/breakhis/BreaKHis_v1/histology_slides/breast/benign/SOB/adenosis
Training set size: 355, Validation set size: 89
Found 1014 images in /content/drive/MyDrive/breakhis/BreaKHis_v1/histology_slides/breast/benign/SOB/fibroadenoma
Training set size: 811, Validation set size: 203
Found 453 images in /content/drive/MyDrive/breakhis/BreaKHis_v1/histology_slides/breast/benign/SOB/phyllodes_tumor
Training set size: 362, Validation set size: 91
Found 569 images in /content/drive/MyDrive/breakhis/BreaKHis_v1/histology_slides/breast/benign/SOB/tubular_adenoma
Training set size: 455, Validation set size: 114
Found 3451 images in /content/drive/MyDrive/breakhis/BreaKHis_v1/histology_slides/breast/malignant/SOB/ductal_carcinoma
Training set size: 2760, Validation set size: 691
Found 626 images in /content/drive/MyDrive/breakhis/BreaKHis_v1/histology_slides/breast/malignant/SOB/lobular_carcinoma
Training set size: 500, Validation set size: 126
Found 792 images

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB5
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, LearningRateScheduler, ReduceLROnPlateau
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import confusion_matrix, classification_report
import numpy as np
import matplotlib.pyplot as plt
import os
import glob
from sklearn.model_selection import train_test_split
import seaborn as sns

# Define base directory
base_dir = '/content/drive/MyDrive/breakhis/BreaKHis_v1/histology_slides/breast'

# Define training and validation directories
train_dir = os.path.join(base_dir, 'train')
val_dir = os.path.join(base_dir, 'validation')

# Benign and malignant subclasses
benign_subclasses = ['adenosis', 'fibroadenoma', 'phyllodes_tumor', 'tubular_adenoma']
malignant_subclasses = ['ductal_carcinoma', 'lobular_carcinoma', 'mucinous_carcinoma', 'papillary_carcinoma']

# Ensure previous work is not lost
# Re-create generators after augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.3,
    zoom_range=0.3,
    horizontal_flip=True,
    rotation_range=40,
    width_shift_range=0.3,
    height_shift_range=0.3,
    brightness_range=[0.9, 1.1],
)

val_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.5)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

validation_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

test_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation',
    shuffle= False
)

# Define the model
model = Sequential([
    EfficientNetB5(input_shape=(224, 224, 3), include_top=False, weights='imagenet'),
    GlobalAveragePooling2D(),
    Dropout(0.8),
    Dense(len(benign_subclasses + malignant_subclasses), activation='softmax', kernel_regularizer=tf.keras.regularizers.l2(0.01))
])

model.compile(optimizer=Adam(learning_rate=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])

# Define callbacks
checkpoint_filepath = '/content/drive/MyDrive/efficientnet_checkpoints/best_model.keras'
checkpoint = ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=False,
    monitor='val_loss',
    mode='min',
    save_best_only=True,
    verbose=1
)

early_stopping = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6)

def scheduler(epoch, lr):
    if epoch < 10:
        return float(lr)
    else:
        return float(lr * tf.math.exp(-0.1))

lr_callback = LearningRateScheduler(scheduler)

# Compute class weights
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(train_generator.classes), y=train_generator.classes)
class_weights = {i: class_weights[i] for i in range(len(class_weights))}

best_test_model_filepath = '/content/drive/MyDrive/efficientnet_checkpoints/best_test_model.keras'

# Define the custom callback to evaluate test accuracy during training
class TestAccuracyCallback(tf.keras.callbacks.Callback):
    def __init__(self, test_generator, save_path):
        super(TestAccuracyCallback, self).__init__()
        self.test_generator = test_generator
        self.best_val_loss = np.inf
        self.save_path = save_path
        self.best_test_accuracy = 0.0  # Initialize the best test accuracy

    def on_epoch_end(self, epoch, logs=None):
        current_val_loss = logs.get('val_loss')
        if current_val_loss < self.best_val_loss:
            self.best_val_loss = current_val_loss
            test_loss, test_accuracy = self.model.evaluate(self.test_generator, verbose=0)
            print(f"\nEpoch {epoch+1}: Test loss = {test_loss}, Test accuracy = {test_accuracy * 100:.2f}%")

            if test_accuracy > self.best_test_accuracy:
                self.best_test_accuracy = test_accuracy

                # Save the model if test accuracy improves
                self.model.save(self.save_path)
                print(f"Best model saved for test set at epoch {epoch+1} with Test accuracy: {test_accuracy * 100:.2f}%")

test_accuracy_callback = TestAccuracyCallback(test_generator, save_path=best_test_model_filepath)


# Train the model
history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=32,
    class_weight=class_weights,
    callbacks=[checkpoint, early_stopping, reduce_lr, lr_callback, test_accuracy_callback]
)

# Load the best model
best_model = tf.keras.models.load_model(checkpoint_filepath)

# Evaluate the best model
loss, accuracy = best_model.evaluate(validation_generator, verbose=0)
print(f'Validation loss: {loss}, Validation accuracy: {accuracy * 100:.2f}%')

plt.figure(figsize=(10, 5))
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend(['Training Accuracy', 'Validation Accuracy'])
plt.show()

# Plot training & validation loss values
plt.figure(figsize=(10, 5))
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend(['Training Loss', 'Validation Loss'])
plt.show()

# Evaluate the best model on the test set
test_loss, test_accuracy = best_model.evaluate(test_generator, verbose=0)
print(f'Test loss: {test_loss}, Test accuracy: {test_accuracy * 100:.2f}%')


# Generate predictions and compute confusion matrix
test_steps = test_generator.samples // test_generator.batch_size
y_pred = best_model.predict(test_generator, steps=test_steps, verbose=1)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = test_generator.classes[:len(y_pred_classes)]

conf_matrix = confusion_matrix(y_true, y_pred_classes)
class_names = list(test_generator.class_indices.keys())

plt.figure(figsize=(12, 8))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names, annot_kws={"size": 14})
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix')
plt.show()

# Print classification report
report = classification_report(y_true, y_pred_classes, target_names=class_names, digits=3)
print(report)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB5
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, LearningRateScheduler, ReduceLROnPlateau
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import confusion_matrix, classification_report
import numpy as np
import matplotlib.pyplot as plt
import os
import glob
from sklearn.model_selection import train_test_split
import seaborn as sns
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import BatchNormalization

# Define base directory
base_dir = '/content/drive/MyDrive/breakhis/BreaKHis_v1/histology_slides/breast'

# Define training and validation directories
train_dir = os.path.join(base_dir, 'train')
val_dir = os.path.join(base_dir, 'validation')

# Benign and malignant subclasses
benign_subclasses = ['adenosis', 'fibroadenoma', 'phyllodes_tumor', 'tubular_adenoma']
malignant_subclasses = ['ductal_carcinoma', 'lobular_carcinoma', 'mucinous_carcinoma', 'papillary_carcinoma']

# Ensure previous work is not lost
# Re-create generators after augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.3,
    zoom_range=0.3,
    horizontal_flip=True,
    rotation_range=40,
    width_shift_range=0.3,
    height_shift_range=0.3,
    brightness_range=[0.9, 1.1],
)

val_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.5)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

validation_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

test_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation',
    shuffle= False
)

'''
# Define the base model (EfficientNetB5) and load pre-trained weights
base_model = EfficientNetB5(input_shape=(224, 224, 3), include_top=False, weights='imagenet')

model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dropout(0.8),
    Dense(1, activation='sigmoid', kernel_regularizer=tf.keras.regularizers.l2(0.01))
])

model.load_weights('/content/drive/MyDrive/efficientnet_checkpoints/best_weights.weights.h5', skip_mismatch=True)
for layer in model.layers:
    if isinstance(layer, BatchNormalization):
        layer.trainable = True  # Fine-tune normalization layers

model.pop()
model.add(Dense(len(benign_subclasses + malignant_subclasses), activation='softmax', kernel_regularizer=tf.keras.regularizers.l2(0.01)))
'''


# Define the model
model = Sequential([
    EfficientNetB5(input_shape=(224, 224, 3), include_top=False, weights='imagenet'),
    GlobalAveragePooling2D(),
    Dropout(0.8),
    Dense(len(benign_subclasses + malignant_subclasses), activation='softmax', kernel_regularizer=tf.keras.regularizers.l2(0.01))
])

# Build the model
model.build((None, 224, 224, 3))

model.load_weights('/content/drive/MyDrive/efficientnet_checkpoints/binary_best_model.keras')


model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# Define callbacks
checkpoint_filepath = '/content/drive/MyDrive/efficientnet_checkpoints/binary_best_model.keras'
checkpoint = ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=False,
    monitor='val_loss',
    mode='min',
    save_best_only=True,
    verbose=1
)

early_stopping = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6)

def scheduler(epoch, lr):
    if epoch < 10:
        return float(lr)
    else:
        return float(lr * tf.math.exp(-0.1))

lr_callback = LearningRateScheduler(scheduler)

# Compute class weights
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(train_generator.classes), y=train_generator.classes)
class_weights = {i: class_weights[i] for i in range(len(class_weights))}


best_test_model_filepath = '/content/drive/MyDrive/efficientnet_checkpoints/binary_best_test_model.keras'

# Define the custom callback to evaluate test accuracy during training
class TestAccuracyCallback(tf.keras.callbacks.Callback):
    def __init__(self, test_generator, save_path):
        super(TestAccuracyCallback, self).__init__()
        self.test_generator = test_generator
        self.best_val_loss = np.inf
        self.save_path = save_path
        self.best_test_accuracy = 0.0  # Initialize the best test accuracy

    def on_epoch_end(self, epoch, logs=None):
        current_val_loss = logs.get('val_loss')
        if current_val_loss < self.best_val_loss:
            self.best_val_loss = current_val_loss
            test_loss, test_accuracy = self.model.evaluate(self.test_generator, verbose=0)
            print(f"\nEpoch {epoch+1}: Test loss = {test_loss}, Test accuracy = {test_accuracy * 100:.2f}%")

            if test_accuracy > self.best_test_accuracy:
                self.best_test_accuracy = test_accuracy

                # Save the model if test accuracy improves
                self.model.save(self.save_path)
                print(f"Best model saved for test set at epoch {epoch+1} with Test accuracy: {test_accuracy * 100:.2f}%")

test_accuracy_callback = TestAccuracyCallback(test_generator, save_path=best_test_model_filepath)

# Train the model
remaining_epochs = 32
history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=remaining_epochs,
    class_weight=class_weights,
    callbacks=[checkpoint, early_stopping, reduce_lr, lr_callback, test_accuracy_callback]
)

# Load the best model

# Evaluate the best model
loss, accuracy = model.evaluate(validation_generator, verbose=0)
print(f'Validation loss: {loss}, Validation accuracy: {accuracy * 100:.2f}%')

plt.figure(figsize=(10, 5))
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend(['Training Accuracy', 'Validation Accuracy'])
plt.show()

# Plot training & validation loss values
plt.figure(figsize=(10, 5))
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend(['Training Loss', 'Validation Loss'])
plt.show()

# Evaluate the best model on the test set
test_loss, test_accuracy = model.evaluate(test_generator, verbose=0)
print(f'Test loss: {test_loss}, Test accuracy: {test_accuracy * 100:.2f}%')


# Generate predictions and compute confusion matrix
#test_steps = test_generator.samples // test_generator.batch_size
#y_pred = best_model.predict(test_generator, steps=test_steps, verbose=1)
y_pred = model.predict(test_generator)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = test_generator.classes[:len(y_pred_classes)]

conf_matrix = confusion_matrix(y_true, y_pred_classes)
class_names = list(test_generator.class_indices.keys())

plt.figure(figsize=(12, 8))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names, annot_kws={"size": 14})
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix')
plt.show()

# Print classification report
report = classification_report(y_true, y_pred_classes, target_names=class_names)
print(report)


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB5
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, LearningRateScheduler, ReduceLROnPlateau
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import confusion_matrix, classification_report
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import glob
from sklearn.model_selection import train_test_split


# Define base directory
base_dir = '/content/drive/MyDrive/breakhis/BreaKHis_v1/histology_slides/breast'

# Define training and validation directories
train_dir = os.path.join(base_dir, 'train')
val_dir = os.path.join(base_dir, 'validation')

# Benign and malignant subclasses
benign_subclasses = ['adenosis', 'fibroadenoma', 'phyllodes_tumor', 'tubular_adenoma']
malignant_subclasses = ['ductal_carcinoma', 'lobular_carcinoma', 'mucinous_carcinoma', 'papillary_carcinoma']

# Data augmentation for training
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    brightness_range=[0.9, 1.1],
)

# For validation and test sets
val_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.5)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

validation_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

test_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation',
    shuffle=False
)

# Define the model for transfer learning
base_model = EfficientNetB5(input_shape=(224, 224, 3), include_top=False, weights='imagenet')

# Freeze the layers of the pre-trained model (except classification head)
for layer in base_model.layers:
    layer.trainable = False

model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dropout(0.8),
    Dense(len(benign_subclasses + malignant_subclasses), activation='softmax', kernel_regularizer=tf.keras.regularizers.l2(0.01))
])

# Load weights from the binary classification model
model.build((None, 224, 224, 3))
base_model.load_weights('/content/drive/MyDrive/efficientnet_checkpoints/binary_best_model.keras', skip_mismatch=True)

model.compile(optimizer=Adam(learning_rate=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])

# Callbacks for training
checkpoint_filepath = '/content/drive/MyDrive/efficientnet_checkpoints/binary_best_model.keras'
checkpoint = ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=False,
    monitor='val_loss',
    mode='min',
    save_best_only=True,
    verbose=1
)

early_stopping = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6)

def scheduler(epoch, lr):
    if epoch < 10:
        return float(lr)
    else:
        return float(lr * tf.math.exp(-0.1))

lr_callback = LearningRateScheduler(scheduler)

# Compute class weights to handle class imbalance
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(train_generator.classes), y=train_generator.classes)
class_weights = {i: class_weights[i] for i in range(len(class_weights))}

# Define the custom callback to evaluate test accuracy during training
class TestAccuracyCallback(tf.keras.callbacks.Callback):
    def __init__(self, test_generator, save_path):
        super(TestAccuracyCallback, self).__init__()
        self.test_generator = test_generator
        self.best_val_loss = np.inf
        self.save_path = save_path
        self.best_test_accuracy = 0.0  # Initialize the best test accuracy

    def on_epoch_end(self, epoch, logs=None):
        current_val_loss = logs.get('val_loss')
        if current_val_loss < self.best_val_loss:
            self.best_val_loss = current_val_loss
            test_loss, test_accuracy = self.model.evaluate(self.test_generator, verbose=0)
            print(f"\nEpoch {epoch+1}: Test loss = {test_loss}, Test accuracy = {test_accuracy * 100:.2f}%")

            if test_accuracy > self.best_test_accuracy:
                self.best_test_accuracy = test_accuracy

                # Save the model if test accuracy improves
                self.model.save(self.save_path)
                print(f"Best model saved for test set at epoch {epoch+1} with Test accuracy: {test_accuracy * 100:.2f}%")

best_test_model_filepath = '/content/drive/MyDrive/efficientnet_checkpoints/binary_best_test_model.keras'
test_accuracy_callback = TestAccuracyCallback(test_generator, save_path=best_test_model_filepath)

# Train the model

# 3. Train the model with the frozen layers first (fine-tuning head)
initial_epochs = 12
history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=initial_epochs,
    class_weight=class_weights,
    callbacks=[checkpoint, early_stopping, reduce_lr, lr_callback, test_accuracy_callback]
)


# 4. Gradual unfreezing - unfreeze part of the base model and recompile with lower LR
for layer in base_model.layers[-20:]:  # Unfreeze last 20 layers as an example
    layer.trainable = True

model.compile(optimizer=Adam(learning_rate=1e-5), loss='categorical_crossentropy', metrics=['accuracy'])

# Continue training with partial unfreezing
fine_tuning_epochs = 20
history_fine_tune = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=fine_tuning_epochs,
    class_weight=class_weights,
    callbacks=[checkpoint, early_stopping, reduce_lr, lr_callback, test_accuracy_callback]
)

# 5. Fully unfreeze and fine-tune all layers
for layer in base_model.layers:
    layer.trainable = True

model.compile(optimizer=Adam(learning_rate=1e-6), loss='categorical_crossentropy', metrics=['accuracy'])

# Final training with all layers unfrozen
final_epochs = 12
history_final = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=final_epochs,
    class_weight=class_weights,
    callbacks=[checkpoint, early_stopping, reduce_lr, lr_callback, test_accuracy_callback]
)
'''
# Plot training and validation accuracy
plt.figure(figsize=(10, 5))
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend(['Training Accuracy', 'Validation Accuracy'])
plt.show()

# Plot training & validation loss
plt.figure(figsize=(10, 5))
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend(['Training Loss', 'Validation Loss'])
plt.show()
'''
# Evaluate the final model on the test set
test_loss, test_accuracy = model.evaluate(test_generator, verbose=0)
print(f'Test loss: {test_loss}, Test accuracy: {test_accuracy * 100:.2f}%')

# Confusion Matrix and Classification Report
y_pred = model.predict(test_generator)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = test_generator.classes[:len(y_pred_classes)]

conf_matrix = confusion_matrix(y_true, y_pred_classes)
class_names = list(test_generator.class_indices.keys())

plt.figure(figsize=(12, 8))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names, annot_kws={"size": 14})
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix')
plt.show()

# Classification report
report = classification_report(y_true, y_pred_classes, target_names=class_names)
print(report)


In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import EfficientNetB5
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
import os
from sklearn.metrics import confusion_matrix, classification_report
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Define base directory for test data
base_dir = '/content/drive/MyDrive/breakhis/BreaKHis_v1/histology_slides/breast'
val_dir = os.path.join(base_dir, 'validation')

benign_subclasses = ['adenosis', 'fibroadenoma', 'phyllodes_tumor', 'tubular_adenoma']
malignant_subclasses = ['ductal_carcinoma', 'lobular_carcinoma', 'mucinous_carcinoma', 'papillary_carcinoma']

# Test data augmentation
val_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255, validation_split=0.5)

test_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation',
    shuffle= False
)

# Define the model
model = Sequential([
    EfficientNetB5(input_shape=(224, 224, 3), include_top=False, weights='imagenet'),
    GlobalAveragePooling2D(),
    Dropout(0.8),
    Dense(len(benign_subclasses + malignant_subclasses), activation='softmax', kernel_regularizer=tf.keras.regularizers.l2(0.01))
])

# Build the model
model.build((None, 224, 224, 3))

model.load_weights('/content/drive/MyDrive/efficientnet_checkpoints/best_test_model.keras')


model.compile(optimizer=Adam(learning_rate=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])



# Evaluate the best model on the test set
test_loss, test_accuracy = model.evaluate(test_generator, verbose=0)
print(f'Test loss: {test_loss}, Test accuracy: {test_accuracy * 100:.2f}%')


# Generate predictions and compute confusion matrix
#test_steps = test_generator.samples // test_generator.batch_size
#y_pred = best_model.predict(test_generator, steps=test_steps, verbose=1)
y_pred = model.predict(test_generator)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = test_generator.classes[:len(y_pred_classes)]

conf_matrix = confusion_matrix(y_true, y_pred_classes)
class_names = list(test_generator.class_indices.keys())

plt.figure(figsize=(12, 8))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names, annot_kws={"size": 14})
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix')
plt.show()

# Print classification report
report = classification_report(y_true, y_pred_classes, target_names=class_names)
print(report)

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import EfficientNetB5
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
import os
from sklearn.metrics import confusion_matrix, classification_report
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

base_dir = '/content/drive/MyDrive/breakhis/BreaKHis_v1/histology_slides/breast'
val_dir = os.path.join(base_dir, 'validation')

benign_subclasses = ['adenosis', 'fibroadenoma', 'phyllodes_tumor', 'tubular_adenoma']
malignant_subclasses = ['ductal_carcinoma', 'lobular_carcinoma', 'mucinous_carcinoma', 'papillary_carcinoma']

val_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255, validation_split=0.5)

test_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation',
    shuffle= False
)

# Define the number of classes (replace with the actual number)
num_classes = len(benign_subclasses + malignant_subclasses)

# Rebuild the original model architecture
def build_model():
    # Load the EfficientNetB5 base model, without the top fully connected layers
    base_model = EfficientNetB5(include_top=False, weights='imagenet', input_shape=(224, 224, 3))

    # Add new layers on top of the base model
    x = base_model.output
    x = GlobalAveragePooling2D()(x)  # Apply global average pooling
    x = Dropout(0.8)(x)  # Add dropout for regularization
    outputs = Dense(num_classes, activation='softmax', kernel_regularizer=tf.keras.regularizers.l2(0.01))(x)

    # Create a new model
    model = Model(inputs=base_model.input, outputs=outputs)

    return model

# Create the model architecture
model = build_model()

# Load the weights from the best model checkpoint
checkpoint_filepath = '/content/drive/MyDrive/efficientnet_checkpoints/best_model1.keras'  # Replace with actual path to your best model checkpoint
model = tf.keras.models.load_model('/content/drive/MyDrive/efficientnet_checkpoints/best_model1.keras')

# Compile the model (you can use the same loss function and metrics as used during training)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(test_generator, verbose=0)
print(f'Test loss: {test_loss}, Test accuracy: {test_accuracy * 100:.2f}%')

# Generate predictions and compute confusion matrix
test_steps = test_generator.samples // test_generator.batch_size
y_pred = model.predict(test_generator, steps=test_steps, verbose=1)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = test_generator.classes[:len(y_pred_classes)]

# Confusion matrix and classification report
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

conf_matrix = confusion_matrix(y_true, y_pred_classes)
class_names = list(test_generator.class_indices.keys())

plt.figure(figsize=(12, 8))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names, annot_kws={"size": 14})
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix')
plt.show()

# Print classification report
report = classification_report(y_true, y_pred_classes, target_names=class_names, digits=3)
print(report)
